## Setup

In [1]:
# --- Preamble
from IPython.display import IFrame
from itertools import groupby 
import networkx as nx
import pandas as pd
import numpy as np
import osmnx as ox
import warnings
import folium
%matplotlib inline

/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# --- Configure settings
ox.settings.useful_tags_path = ["bridge","tunnel","oneway","lanes","ref","name","highway","maxspeed","service","access","area","landuse","width","est_width","junction","surface","tracktype","natural","landuse"] # To make OSMNX output the information we need to determine paved status
ox.settings.useful_tags_way = ["bridge","tunnel","oneway","lanes","ref","name","highway","maxspeed","service","access","area","landuse","width","est_width","junction","surface","tracktype","natural","landuse"] # To make OSMNX output the information we need to determine paved status
paved_types = ["primary", "unclassified", "tertiary", "residential", "secondary", "service"] # Highway types that are paved

## Loading GPX track of hiking trail

In [3]:
# --- Loading the GPX track
# Define the range of GPX points to process in the current batch
n1 = 0
n2 = 100
# n1 = 200
# n2 = 300
# n1 = 2115 # First point
# n2 = 2119 # Last point
# n1 = 2100 # First point
# n2 = 2200 # Last point
# n1 = 1100
# n2 = 1200

# Load and process
# trailname = 'gr16'
trailname = 'gr131'
hike = pd.read_csv('data/' + trailname + '.csv')
coords = hike.loc[n1:n2][['latitude','longitude']].values.tolist()

# Calculate the bounding box around points n1 through n2
delta = 0.005 # Tolerance in degrees on the bounding box
lat_min = hike.loc[n1:n2]['latitude'].min()
lat_max = hike.loc[n1:n2]['latitude'].max()
lon_min = hike.loc[n1:n2]['longitude'].min()
lon_max = hike.loc[n1:n2]['longitude'].max()

## Constructing OSM street network

In [4]:
# --- Download the street network based on bounding box
G = ox.graph_from_bbox(lat_max+delta,
                       lat_min-delta,
                       lon_max+delta,
                       lon_min-delta,
                       network_type="all_private", clean_periphery=False)

/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/osmnx/graph.py:488: UserWarning: the graph-level street_count attribute will likely be inaccurate when you set clean_periphery=False
  warnings.warn(msg)


In [5]:
# --- Processing the street network
network_points, network_edges = ox.graph_to_gdfs(G) # Convert the street network
network_points.sort_index(inplace=True) # Sort the nodes for faster selections with .loc
network_edges.sort_index(inplace=True) # Sort the edges for faster selections with .loc
nodes = network_points[['y','x']].values.tolist() # Vector with [lat, lon] of all network nodes

In [6]:
# Color pattern
# Black: raw street network note
# Red: GPX track node
# White: node_list
# Blue: route_list
# Green: segment_list

# Plotting the GPX track
id_focus = round((n2-n1+1)/2)
chart0 = folium.Map(location=coords[id_focus], zoom_start=12, tiles="OpenStreetMap") # Map setup
# Adding GPX track
newline = folium.PolyLine(locations=coords, popup='None', color='red', weight=3)
newline.add_to(chart0)
# Render all nodes
# for index in network_points.index:
#     newmarker = folium.CircleMarker(location=[network_points.loc[index]['y'],network_points.loc[index]['x']],radius=1,color='black')
#     newmarker.add_to(chart0)
# Draw bounding box
delta = 0.005
bbox = [[lat_min-delta,lon_min-delta],
        [lat_min-delta,lon_max+delta],
        [lat_max+delta,lon_max+delta],
        [lat_max+delta,lon_min-delta],
        [lat_min-delta,lon_min-delta]]
newline = folium.PolyLine(locations=bbox, popup='None', color='blue', weight=2)
newline.add_to(chart0)

In [7]:
# Render the map
filepath = "data/chart0.html"
chart0.save(filepath)
IFrame(filepath, width=600, height=500)

## Map matching GPX track to OSM network

In [8]:
# --- Defining some functions
# This function computes the distances between point-node1 and point-node2
def get_distances(node1,node2,point):
    R = 6371.8 # Mean earth radius [km]
    # Converting degrees to radians
    lon1 = np.radians(node1[1])
    lat1 = np.radians(node1[0])
    lon2 = np.radians(node2[1])
    lat2 = np.radians(node2[0])
    lat  = np.radians(point[0])
    lon  = np.radians(point[1])
    # Computing distances between point-node1 and point-node2
    r1 = [R*np.cos(lat1)*np.cos(lon1), R*np.cos(lat1)*np.sin(lon1), R*np.sin(lat1)]
    r2 = [R*np.cos(lat2)*np.cos(lon2), R*np.cos(lat2)*np.sin(lon2), R*np.sin(lat2)]
    r  = [R*np.cos(lat)*np.cos(lon),   R*np.cos(lat)*np.sin(lon),   R*np.sin(lat)]
    dr1 = [r1[0]-r[0], r1[1]-r[1], r1[2]-r[2]]
    dr2 = [r2[0]-r[0], r2[1]-r[1], r2[2]-r[2]]
    d1 = np.sqrt(dr1[0]*dr1[0] + dr1[1]*dr1[1] + dr1[2]*dr1[2])
    d2 = np.sqrt(dr2[0]*dr2[0] + dr2[1]*dr2[1] + dr2[2]*dr2[2])
    return d1,d2

# This function returns a list with the [lat, lon] of each point within an OSM edge
def edge_to_coords(network_edges,edge_id):
    lon = list(network_edges.loc[(edge_id[0],edge_id[1])]['geometry'][0].coords.xy[0])
    lat = list(network_edges.loc[(edge_id[0],edge_id[1])]['geometry'][0].coords.xy[1])
    edge_coords = list(zip(lon,lat))
    return [[coord[1],coord[0]] for coord in edge_coords]

In [9]:
# Map matching is done as follows, for each point of the GPX track:
# Step 1: Determine which edge the point is closest to
# Step 2: Determine whether the point is closer to the start/end of that edge
# Step 3: Store the node ID of start/end location in node_list
# Step 4: Remove successive duplicates from the node_list

# Setup
k = 0 # To display where we are in the loop
node_list_raw = [] # Will contain the nodes between which pathfinding should take place
warnings.filterwarnings("ignore", category=UserWarning) # TODO: Figure out projection issue so this warning is not thrown

for point in coords:
    
    txtstring = '\r' + 'Handling GPX point ' + str(k) + ' of ' + str(len(coords)-1)
    print(txtstring, end='\r', flush=True)
    
    # Step 1: Determine which edge this GPX point is closest to
    ox_nearest = ox.distance.nearest_edges(G, point[1], point[0]) # OSMNX call to find the nearest edge, returns a tuple
    nearest_edge = [ox_nearest[0], ox_nearest[1]] # 
    nearest_edge_coords = edge_to_coords(network_edges, nearest_edge) # Get coordinates of the points that make up that edge
    
    # Step 2: Determine whether this GPX point is closer to the start/end of the edge
    vertex_start = nearest_edge_coords[0] # Start vertex coords of the edge
    vertex_end   = nearest_edge_coords[-1] # End vertex coords of the edge
    d_start, d_end = get_distances(vertex_start, vertex_end, point) # Distance point->vertex_start and point->vertex_end
    
    # Step 3: Store the ID of the nearest vertex in node_list_raw
    if d_start<d_end:
        node_list_raw.append(nearest_edge[0])
    else:
        node_list_raw.append(nearest_edge[1])
        
    k += 1 # Increment counter
    
# Step 4: Remove successive duplicates from the node list
node_list = [node[0] for node in groupby(node_list_raw)]

In [10]:
# Plotting the node_list VS the GPX points
id_focus = round((n2-n1+1)/2)
chart1 = folium.Map(location=coords[id_focus], zoom_start=13, tiles="OpenStreetMap") # Map setup
# Adding GPX track
newline = folium.PolyLine(locations=coords, popup='None', color='red', weight=1)
newline.add_to(chart1)
# Adding GPX points
for vertex in coords:
    newmarker = folium.CircleMarker(location=vertex,radius=1,color='red')
    newmarker.add_to(chart1)
# Adding(pruned) node_list
for node in node_list:
    full_node = network_points.loc[node]
    vertex = [full_node['y'],full_node['x']]
    newmarker = folium.CircleMarker(location=vertex,radius=3,color='black',popup=str(node))
    newmarker.add_to(chart1)
# Adding RAW node list and mapping lines to GPX points
# for i in range(len(node_list_raw)):
#     vertex_track = coords[i]
#     vertex_mapped = [network_points.loc[node_list_raw[i]]['y'],network_points.loc[node_list_raw[i]]['x']]
#     newmarker = folium.CircleMarker(location=vertex_track,radius=1,color='red')
#     newmarker.add_to(chart1)
#     newmarker = folium.CircleMarker(location=vertex_mapped,radius=1,color='white')
#     newmarker.add_to(chart1)
#     newline = folium.PolyLine(locations=[vertex_track,vertex_mapped], popup='None', color='black', weight=1)
#     newline.add_to(chart1)

In [11]:
# Render the map
filepath = "data/chart1.html"
chart1.save(filepath)
IFrame(filepath, width=600, height=500)

## Constructing a path between the matched OSM nodes

In [12]:
# Path construction is done with the following steps
# Step 1: Use OSMNX to generate the shortest walking route between each node pair in node_list

# Setup
route_list_raw = [] # Will contain IDs of all nodes that make up the shortest route between the nodes in node_list

# Step 1: Use OSMNX to generate the shortest walking route between each node pair in node_list, store the corresponding nodes in route_list
for i in range(0,len(node_list)-1):
    
#     print(f'Handling node_list point #{i} of {len(node_list)-1}')
    
    # Extracting information about the nodes
    id_start     = node_list[i] # ID of the start node
    id_end       = node_list[i+1] # ID of the end node
    node_start   = network_points.loc[id_start] # Start node with relevant information
    node_end     = network_points.loc[id_end] # End node with relevant information
    vertex_start = [node_start['y'],node_start['x']] # Coordinates of start node
    vertex_end   = [node_end['y'],node_end['x']] # Coordinates of end node
    
#     print(f'Calculating shortest route between nodes {id_start} and {id_end}')
    
    # Pathfinding between node_start and node_end
    route1 = ox.shortest_path(G, node_list[i], node_list[i+1]) # Route from node_start -> node_end
    route2 = ox.shortest_path(G, node_list[i+1], node_list[i]) # Route from node_end -> node_start
    
    # Select the direction that results in the shortest path (we do this to deal with one-way streets)
    if len(route1)<len(route2):
        route_list_raw.extend(route1)
    else:
        route_list_raw.extend(route2[::-1]) # We flip route2 because it runs opposite to the GPX track
        
# Remove successive duplicates from the route_list
route_list = [node[0] for node in groupby(route_list_raw)]

In [13]:
# Plotting the route_list vs. node_list
id_focus = round((n2-n1+1)/2)
chart2 = folium.Map(location=coords[id_focus], zoom_start=13, tiles="OpenStreetMap") # Map setup

# --- Adding route_list
for i in range(len(route_list)-1): # Drawing a line
    id_start     = route_list[i] # ID of the start node
    id_end       = route_list[i+1] # ID of the end node
    node_start   = network_points.loc[id_start] # Start node with relevant information
    node_end     = network_points.loc[id_end] # End node with relevant information
    vertex_start = [node_start['y'],node_start['x']] # Coordinates of start node
    vertex_end   = [node_end['y'],node_end['x']] # Coordinates of end node
    newline = folium.PolyLine(locations=[vertex_start,vertex_end], color='blue', weight=1, popup='None')
    newline.add_to(chart2)
k = 0
for node in route_list: # Drawing the markers
    full_node = network_points.loc[node]
    vertex = [full_node['y'],full_node['x']]
    newmarker = folium.CircleMarker(location=vertex,radius=2,color='blue',popup=str(node))
    newmarker.add_to(chart2)
    k += 1

# --- Adding node_list
k = 0
for node in node_list:
    full_node = network_points.loc[node]
    vertex = [full_node['y'],full_node['x']]
    newmarker = folium.CircleMarker(location=vertex,radius=1,color='red',popup=str(k))
    newmarker.add_to(chart2)
    k += 1

In [14]:
# --- Render the map
filepath = "data/chart2.html"
chart2.save(filepath)
IFrame(filepath, width=600, height=500)

## Extracting the path segments

In [15]:
def cartesian_distance(lat0,lon0,lat1,lon1):
    R = 6371.8 # Mean earth radius [km]
    lat0r = np.radians(lat0)
    lon0r = np.radians(lon0)
    lat1r = np.radians(lat1)
    lon1r = np.radians(lon1)
    
    r1 = [R*np.cos(lat0r)*np.cos(lon0r), R*np.cos(lat0r)*np.sin(lon0r), R*np.sin(lat0r)]
    r2 = [R*np.cos(lat1r)*np.cos(lon1r), R*np.cos(lat1r)*np.sin(lon1r), R*np.sin(lat1r)]
    dr = [r1[0] - r2[0], r1[1] - r2[1], r1[2] - r2[2]]
    return 1000.0*np.sqrt(dr[0]*dr[0] + dr[1]*dr[1] + dr[2]*dr[2])

In [16]:
# Segment extraction is done with the following steps
# Step 1: Extract the coordinates of all segments within each node pair from route_list
# Step 2: Merge the coordinates and additional information into the segment_list
# Step 3: Remove any segments that are traversed twice
# Step 4: Post-process the length and paved status of the entire route

# Setup
segment_list_raw = [] # Will contain all OSM edge segments that were matched to the GPX track [x0 y0 x1 y1 d dcum highway surface paved]

for i in range(0,len(route_list)-1): # Loop over all pairs in the route_list
    
#     print(f'Handling route_list section #{i} of {len(route_list)-1}')
    
    # Extracting information about the nodes
    id_start     = route_list[i] # ID of the start node
    id_end       = route_list[i+1] # ID of the end node
    node_start   = network_points.loc[id_start] # Start node with relevant information
    node_end     = network_points.loc[id_end] # End node with relevant information
    vertex_start = [node_start['y'],node_start['x']] # Coordinates of start node
    vertex_end   = [node_end['y'],node_end['x']] # Coordinates of end node
    
#     print(f'   It connects nodes {id_start} and {id_end}')
    
    # Selecting the corresponding edge
    edge_id = [id_start, id_end]
    try: # To deal with one-way streets
        edge = network_edges.loc[(id_start, id_end)] # Get coordinates of the points that make up that edge
        edge_coords = edge_to_coords(network_edges, [id_start, id_end])
    except:
        edge = network_edges.loc[(id_end, id_start)] # Get coordinates of the points that make up that edge
        edge_coords = edge_to_coords(network_edges, [id_end, id_start])
        edge_coords = edge_coords[::-1]
        
    # Filling the segment_list_raw list
    d_cart_sum = 0
    d_osm_sum = 0
    for j in range(0,len(edge_coords)-1): # Loop over all vertex pairs in the edge        
        x0 = edge_coords[j][0]
        y0 = edge_coords[j][1]
        x1 = edge_coords[j+1][0]
        y1 = edge_coords[j+1][1]
        d_cart = cartesian_distance(x0,y0,x1,y1)
#         print(edge)
        d_osm = edge['length'][0]/(len(edge_coords)-1) # just divide the length equally between segments
        highway = edge['highway'][0]
        surface = None
        tracktype = None
        if 'surface' in edge.columns:
            surface = edge['surface'][0]
        if 'tracktype' in edge.columns:
            tracktype = edge['tracktype'][0]
        newline = [x0,y0,x1,y1,d_cart,d_osm,highway,surface,tracktype]
        segment_list_raw.append(newline)
        d_cart_sum += d_cart
        d_osm_sum += d_osm

In [17]:
# Plotting the segment_list VS the route_list
id_focus = round((n2-n1+1)/2)
chart3 = folium.Map(location=coords[id_focus], zoom_start=12, tiles="OpenStreetMap") # Map setup

# --- Adding route_list
for i in range(len(route_list)-1): # Drawing a line
    id_start     = route_list[i] # ID of the start node
    id_end       = route_list[i+1] # ID of the end node
    node_start   = network_points.loc[id_start] # Start node with relevant information
    node_end     = network_points.loc[id_end] # End node with relevant information
    vertex_start = [node_start['y'],node_start['x']] # Coordinates of start node
    vertex_end   = [node_end['y'],node_end['x']] # Coordinates of end node
    newline = folium.PolyLine(locations=[vertex_start,vertex_end], color='blue', weight=1, popup='None')
    newline.add_to(chart3)
k = 0
for node in route_list: # Drawing the markers
    full_node = network_points.loc[node]
    vertex = [full_node['y'],full_node['x']]
    newmarker = folium.CircleMarker(location=vertex,radius=3,color='blue',popup=str(k))
    newmarker.add_to(chart3)
    k += 1
    
# --- Adding segment_list
for segment in segment_list_raw: # Drawing a line
    vertex_start = segment[0:2]
    vertex_end = segment[2:4]
    newline = folium.PolyLine(locations=[vertex_start,vertex_end], color='purple', weight=3, popup='None')
    newline.add_to(chart3)
k = 0
for segment in segment_list_raw: # Drawing the markers
    vertex = segment[0:2]
    newmarker = folium.CircleMarker(location=vertex,radius=1,color='purple',popup=str(k))
    newmarker.add_to(chart3)
    k += 1

In [18]:
# Render the map
filepath = "data/chart3.html"
chart3.save(filepath)
IFrame(filepath, width=600, height=500)

## Comparing the calculated path with the original GPX track

In [19]:
# Plotting the segment_list VS the route_list
id_focus = round((n2-n1+1)/2)
chart5 = folium.Map(location=coords[id_focus], zoom_start=12, tiles="OpenStreetMap") # Map setup

# Adding GPX track
newline = folium.PolyLine(locations=coords, popup='None', color='red', weight=2)
newline.add_to(chart5)

# --- Adding segment_list
for segment in segment_list_raw: # Drawing a line
    vertex_start = segment[0:2]
    vertex_end = segment[2:4]
    newline = folium.PolyLine(locations=[vertex_start,vertex_end], color='black', weight=2, popup='None')
    newline.add_to(chart5)

In [20]:
# Render the map
filepath = "data/chart5.html"
chart5.save(filepath)
IFrame(filepath, width=600, height=500)